<a href="https://colab.research.google.com/github/HromovAndrey/II_mood_3_part_2_practic/blob/Pratica/_____2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [113]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/Used%20Car%20Dataset.csv", index_col="Unnamed: 0")
df.drop(columns=['car_name', 'registration_year', 'ownsership', 'transmission'], inplace=True)

Дані про ціни вживаних автомобілів у Індії
* car_name - назва авто у форматі 'рік компанія основна назва'
* registration_year - місяць та рік реєстрації
* ownsership - кількість власників у форматі 'Second owner'
* transmission - тип коробки передач
* insurance_validity - тип страховки
* fuel_type - вид палива
* seats - кількість сидінь
* kms_driven - пробіг
* manufacturing_year - рік виробництва, має тип object, потрібно застосувати astype(int)
* mileage(kmpl) - скільки кілометрів проїде за літр палива
* engine(cc) - об'єм двигуна у мл
* max_power(bhp) - потужність у кінських силах
* torque(Nm) - крутний момент двигуна
* price(in lakhs) - ціна у сто тисяч рупій

[Повний аналіз даних](https://www.kaggle.com/code/abdelrasoul/used-cars-prices-prediction)

In [114]:
df.head()

,insurance_validity,fuel_type,seats,kms_driven,manufacturing_year,mileage(kmpl),engine(cc),max_power(bhp),torque(Nm),price(in lakhs)
0,Comprehensive,Petrol,5,56000,2017,7.81,2996.0,2996.0,333.0,63.75
1,Comprehensive,Petrol,5,30615,2020,17.40,999.0,999.0,9863.0,8.99
2,Comprehensive,Diesel,5,24000,2018,20.68,1995.0,1995.0,188.0,23.75
3,Comprehensive,Petrol,5,18378,2019,16.50,1353.0,1353.0,13808.0,13.56
4,Comprehensive,Petrol,5,44900,2019,14.67,1798.0,1798.0,17746.0,24.00


In [115]:
df = df[['fuel_type', 'kms_driven', 'engine(cc)', 'price(in lakhs)']]
df.head()

,fuel_type,kms_driven,engine(cc),price(in lakhs)
0,Petrol,56000,2996.0,63.75
1,Petrol,30615,999.0,8.99
2,Diesel,24000,1995.0,23.75
3,Petrol,18378,1353.0,13.56
4,Petrol,44900,1798.0,24.00


# Завдання 1
Виведіть основні числові характеристики даних, розділіть на тренувальну та тестову чатини

In [116]:
df.describe()

,kms_driven,engine(cc),price(in lakhs)
count,1553.000000,1.550000e+03,1553.000000
mean,52841.931101,1.471857e+10,166.141494
std,40067.800347,2.185629e+11,3478.855090
min,620.000000,5.000000e+00,1.000000
25%,30000.000000,1.197000e+03,4.660000
50%,49134.000000,1.462000e+03,7.140000
75%,70000.000000,1.995000e+03,17.000000
max,810000.000000,3.258640e+12,95000.000000


In [117]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print(train_df.head())
print(test_df.head())

     fuel_type  kms_driven  engine(cc)  price(in lakhs)
1289    Petrol       32997       796.0             3.39
1233    Diesel       32141       885.0             6.00
1208    Diesel       86001      1498.0             5.75
1253    Petrol       92000      1198.0             3.60
382     Petrol       42346       998.0             3.01
     fuel_type  kms_driven  engine(cc)  price(in lakhs)
1522    Petrol        7643      1197.0             6.19
270     Petrol       63249      1197.0             4.59
353     Diesel       68000      2393.0            14.25
354     Petrol       52565      1198.0             4.17
1050    Diesel       70000      1950.0            39.95


# Завдання 2
Створіть Pipeline для обробки числових стовпчиків(заповнення пропусків)

In [118]:
numeric_features = ['kms_driven', 'engine(cc)']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])
print(numeric_transformer)


Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])


# Завдання 3
Створіть Pipeline для обробки категоріальних стовпчиків(заповнення пропусків та кодування)

In [119]:
categorical_features = ['fuel_type']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

print(categorical_transformer)


Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))])


# Завдання 4
Об'єднайте попередні Pipeline в один ColumnTransformer

In [120]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

print(preprocessor)


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 ['kms_driven', 'engine(cc)']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['fuel_type'])])


# Завдання 5
Створіть остаточну модель та натренуйте її

In [121]:

model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])



In [122]:
X_train = train_df.drop('price(in lakhs)', axis=1)
y_train = train_df['price(in lakhs)']
X_test = test_df.drop('price(in lakhs)', axis=1)
y_test = test_df['price(in lakhs)']

In [123]:
model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['kms_driven', 'engine(cc)']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['fuel_type'])])),
                ('regressor', LinearRegression())])

# Завдання 6
Виведіть основні метрики моделі

In [124]:
y_pred = model.predict(X_test)

In [125]:
print(f"Mean Squared Error: {mean_squared_error(y_test, y_pred)}")
print(f"R^2 Score: {r2_score(y_test, y_pred)}")

Mean Squared Error: 28936574.498962827
R^2 Score: -0.0006668163024980966


# Завдання 7
Збережіть модель

In [126]:
joblib.dump(model, 'used_car_price_model.pkl')

['used_car_price_model.pkl']

In [127]:
loaded_model = joblib.load('used_car_price_model.pkl')

In [128]:
data = pd.DataFrame({'fuel_type': ['Diesel'],
                     'kms_driven': [48000],
                     'engine(cc)': [1832]})

data

,fuel_type,kms_driven,engine(cc)
0,Diesel,48000,1832


In [129]:
predicted_price = loaded_model.predict(data)
print(f"Прогнозована ціна: {predicted_price[0]} лакхів")

Прогнозована ціна: 1.0528001298786904 лакхів
